# About this notebook

This notebook is just used to quickly demonstrate some basic functionality of PHIDL.  For a much more expansive tutorial with more thorough explanations, please see the phidl_tutorial_example.py file (link here: [phidl_tutorial_example.py](https://github.com/amccaugh/phidl/blob/master/phidl/phidl_tutorial_example.py#L26) )


In [ ]:
%matplotlib notebook
from phidl import Device
from phidl import quickplot as qp # Rename "quickplot()" to the easier "qp()"
import phidl.geometry as pg


# Create a blank device (essentially an empty GDS cell with some special features)
D = Device('mydevice')

# Create and add a polygon from separate lists of x points and y points
# e.g. [(x1, x2, x3, ...), (y1, y2, y3, ...)]
poly1 = D.add_polygon( [(8,6,7,9), (6,8,9,5)] )

# Alternatively, create and add a polygon from a list of points
# e.g. [(x1,y1), (x2,y2), (x3,y3), ...] using the same function
poly2 = D.add_polygon( [(0, 0), (1, 1), (1, 3), (-3, 3)] )

qp(D) # quickplot it!


In [ ]:
b